In [ ]:
import typing

from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle as mpl_Rectangle
from matplotlib.ticker import MultipleLocator as mpl_MultipleLocator
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp

from pylib.hanoi import get_hanoi_value_at_index


In [ ]:
nbm.print_metadata()


In [ ]:
def lolliplot(
    data: pd.DataFrame,
    x: str,
    y: str,
    hue: str,
    stop: typing.Optional[int] = None,
    **kwargs
) -> plt.Axes:
    ax = sns.barplot(
        data=data,
        x=x,
        y=y,
        color="k",
        gap=0.6,
        **kwargs,
    )
    sns.scatterplot(
        data=data,
        x=x,
        y=y,
        hue=hue,
        palette="tab10",
        legend=False,
        **{
            "ax": ax,
            **kwargs,
        },
    )
    ax.yaxis.set_major_locator(mpl_MultipleLocator(1))
    ax.spines[["right", "top"]].set_visible(False)
    ax.set_xlabel("Time ($T$)")

    if stop is not None:
        ax.set_xticks([0, stop])
        ax.set_xticks(np.arange(0, stop, 4), minor=True)
        ax.set_xticklabels([0, stop])

    return ax


## Generate Hanoi Data


In [ ]:
records = [
    {
        "index": i,
        "hanoi value": get_hanoi_value_at_index(i),
    }
    for i in range(200)
]
df = pd.DataFrame.from_records(records)


## Plot Hanoi Data


In [ ]:
aspect = 2.5
height = 1.6
plt.rcParams["font.size"] = 12


In [ ]:
for lim in 16, 32:
    for xlim in None, 32:
        outattrs = {"what": f"hanoi-{lim}"}
        if xlim is not None:
            outattrs["xlim"] = xlim
        saveit, ax = tp.tee(
            lolliplot,
            df[df["index"] < lim],
            x="index",
            y="hanoi value",
            hue="hanoi value",
            stop=lim,
            teeplot_callback=True,
            teeplot_outattrs=outattrs,
            teeplot_subdir="hanoi-strategy-small-epoch",
        )

        ax.set_ylim(-0.66, 6.66)
        ax.figure.set_figheight(height)
        ax.set_aspect(aspect)

        if xlim is not None:  # set xlim as-if to account for padding
            lolliplot(
                data=df[df["index"] < xlim],
                x="index",
                y="hanoi value",
                hue="hanoi value",
                stop=lim,
                ax=ax,
                alpha=0.0,
            )

        saveit()


## Illustrate Top N Strategy


In [ ]:
for xlim in None, 32:
    outattrs = {"what": "top-n-before"}
    if xlim is not None:
        outattrs["xlim"] = xlim

    saveit, ax = tp.tee(
        lolliplot,
        df[df["index"] < 16],
        x="index",
        y="hanoi value",
        hue="hanoi value",
        stop=16,
        teeplot_callback=True,
        teeplot_outattrs=outattrs,
        teeplot_subdir="hanoi-strategy-small-epoch",
    )
    if xlim is not None:  # set xlim as-if to account for padding
        lolliplot(
            data=df[df["index"] < xlim],
            x="index",
            y="hanoi value",
            hue="hanoi value",
            ax=ax,
            alpha=0.0,
        )

    ax.set_ylim(-0.66, 5.66)
    ax.figure.set_figheight(height)
    ax.set_aspect(aspect)
    xlim_ = ax.get_xlim()  # prevent rectangles from skewing padding

    for y in -0.5, 0.5, 1.5, 2.5, 3.5:
        for fill, alpha, hatch in ("green", 0.1, ""), (None, 0.5, ""):
            rect = mpl_Rectangle(
                (-0.5, y),
                width=17,
                height=1,
                alpha=alpha,
                color="green",
                fill=fill,
                hatch=hatch,
                zorder=10,
            )
            ax.add_patch(rect)

    ax.set_xlim(xlim_)
    saveit()


In [ ]:
outattrs = {"what": "top-n-after"}

saveit, ax = tp.tee(
    lolliplot,
    df[df["index"] < 32],
    x="index",
    y="hanoi value",
    hue="hanoi value",
    stop=32,
    teeplot_callback=True,
    teeplot_outattrs=outattrs,
    teeplot_subdir="hanoi-strategy-small-epoch",
)

ax.set_ylim(-0.66, 5.66)
ax.figure.set_figheight(height)
ax.set_aspect(aspect)
xlim_ = ax.get_xlim()  # prevent rectangles from skewing padding

for y in 0.5, 1.5, 2.5, 3.5, 4.5:
    for fill, alpha, hatch in ("green", 0.1, ""), (None, 0.5, ""):
        rect = mpl_Rectangle(
            (-0.5, y),
            width=33,
            height=1,
            alpha=alpha,
            color="green",
            fill=fill,
            hatch=hatch,
            zorder=10,
        )
        ax.add_patch(rect)

ax.set_xlim(xlim_)
saveit()


## Illustrate Last N Strategy


In [ ]:
for xlim in None, 32:
    outattrs = {"what": "last-n-before"}
    if xlim is not None:
        outattrs["xlim"] = xlim

    saveit, ax = tp.tee(
        lolliplot,
        df[df["index"] < 16],
        x="index",
        y="hanoi value",
        hue="hanoi value",
        stop=16 ,
        teeplot_callback=True,
        teeplot_outattrs=outattrs,
        teeplot_subdir="hanoi-strategy-small-epoch",
    )
    if xlim is not None:  # set xlim as-if to account for padding
        lolliplot(
            data=df[df["index"] < xlim],
            x="index",
            y="hanoi value",
            hue="hanoi value",
            ax=ax,
            alpha=0.0,
        )

    ax.set_ylim(-0.66, 5.66)
    ax.figure.set_figheight(height)
    ax.set_aspect(aspect)
    xlim_ = ax.get_xlim()  # prevent rectangles from skewing padding

    for y, dx in (0, 17), (1, 17), (2, 17), (3, 17), (4, 17):
        rect = mpl_Rectangle(
            (16 - dx, y - 0.5),
            width=dx,
            height=1,
            alpha=0.1,
            color="green",
            zorder=10,
        )
        ax.add_patch(rect)
        rect = mpl_Rectangle(
            (16 - dx, y - 0.5),
            width=dx,
            height=1,
            alpha=0.5,
            color="green",
            fill=None,
            zorder=10,
        )
        ax.add_patch(rect)

    ax.set_xlim(xlim_)
    saveit()


In [ ]:
outattrs = {"what": "last-n-after"}

saveit, ax = tp.tee(
    lolliplot,
    df[df["index"] < 32],
    x="index",
    y="hanoi value",
    hue="hanoi value",
    stop=32,
    teeplot_callback=True,
    teeplot_outattrs=outattrs,
    teeplot_subdir="hanoi-strategy-small-epoch",
)

ax.set_ylim(-0.66, 5.66)
ax.figure.set_figheight(height)
ax.set_aspect(aspect)
xlim_ = ax.get_xlim()  # prevent rectangles from skewing padding

for y, dx in (0, 9), (1, 17), (2, 33), (3, 33), (4, 33), (5, 33):
    rect = mpl_Rectangle(
        (32 - dx, y - 0.5),
        width=dx,
        height=1,
        alpha=0.1,
        color="green",
        zorder=10,
    )
    ax.add_patch(rect)
    rect = mpl_Rectangle(
        (32 - dx, y - 0.5),
        width=dx,
        height=1,
        alpha=0.5,
        color="green",
        fill=None,
        zorder=10,
    )
    ax.add_patch(rect)

ax.set_xlim(xlim_)
saveit()


## Illustrate First N Strategy


In [ ]:
for xlim in None, 32:
    outattrs = {"what": "first-n-before"}
    if xlim is not None:
        outattrs["xlim"] = xlim
    saveit, ax = tp.tee(
        lolliplot,
        df[df["index"] < 16],
        x="index",
        y="hanoi value",
        hue="hanoi value",
        stop=16,
        teeplot_callback=True,
        teeplot_outattrs=outattrs,
        teeplot_subdir="hanoi-strategy-small-epoch",
    )
    if xlim is not None:  # set xlim as-if to account for padding
        lolliplot(
            data=df[df["index"] < xlim],
            x="index",
            y="hanoi value",
            hue="hanoi value",
            ax=ax,
            alpha=0.0,
        )

    ax.set_ylim(-0.66, 5.66)
    ax.figure.set_figheight(height)
    ax.set_aspect(aspect)
    xlim_ = ax.get_xlim()  # prevent rectangles from skewing padding


    for y, dx in (0, 17), (1, 17), (2, 17), (3, 17), (4, 17):
        rect = mpl_Rectangle(
            (-1, y - 0.5),
            width=dx,
            height=1,
            alpha=0.1,
            color="green",
            zorder=10,
        )
        ax.add_patch(rect)
        rect = mpl_Rectangle(
            (-1, y - 0.5),
            width=dx,
            height=1,
            alpha=0.5,
            color="green",
            fill=None,
            zorder=10,
        )
        ax.add_patch(rect)

    ax.set_xlim(xlim_)
    saveit()


In [ ]:
outattrs = {"what": "first-n-after"}

saveit, ax = tp.tee(
    lolliplot,
    df[df["index"] < 32],
    x="index",
    y="hanoi value",
    hue="hanoi value",
    stop=32,
    teeplot_callback=True,
    teeplot_outattrs=outattrs,
    teeplot_subdir="hanoi-strategy-small-epoch",
)

ax.set_ylim(-0.66, 5.66)
ax.figure.set_figheight(height)
ax.set_aspect(aspect)
xlim_ = ax.get_xlim()  # prevent rectangles from skewing padding

for y, dx in (0, 8), (1, 16), (2, 33), (3, 33), (4, 33), (5, 33):
    rect = mpl_Rectangle(
        (-1, y - 0.5),
        width=dx,
        height=1,
        alpha=0.1,
        color="green",
        zorder=10,
    )
    ax.add_patch(rect)
    rect = mpl_Rectangle(
        (-1, y - 0.5),
        width=dx,
        height=1,
        alpha=0.5,
        color="green",
        fill=None,
        zorder=10,
    )
    ax.add_patch(rect)

ax.set_xlim(xlim_)
saveit()
